In [1]:
# Import required libraries
import sys
from pathlib import Path
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import stats

# Set working directory
notebook_dir = Path.cwd()
if notebook_dir.name != 'convolution_HRF':
    convolution_hrf_dir = (
        notebook_dir / 'convolution_HRF' 
        if (notebook_dir / 'convolution_HRF').exists() 
        else notebook_dir.parent / 'convolution_HRF'
    )
    os.chdir(convolution_hrf_dir)
    print(f"Changed working directory to: {convolution_hrf_dir}")
else:
    print(f"Working directory: {notebook_dir}")

# Add to path
sys.path.insert(0, str(Path.cwd()))

# Import correlation analysis functions
from correlation_analysis import (
    load_extracted_data,
    calculate_within_bez_correlations,
    calculate_cochlea_bez_correlation,
    calculate_percentile_rank,
    analyze_correlations,
    plot_correlation_comparison
)

print("✓ All modules imported successfully")

Working directory: /home/ekim/PycharmProjects/phd_firstyear/subcorticalSTRF/convolution_HRF
✓ All modules imported successfully


## Step 1: Configure File Paths

Specify the paths to the extracted BEZ and Cochlea data files.
These should be at the same dB level for meaningful comparison.

In [ ]:
# File paths for extracted data (adjust these to your actual files)
bez_file = 'results/extracted/BEZ_extracted_60.0dB_20251027_155115.pkl'
cochlea_file = 'results/extracted/COCHLEA_extracted_60.0dB_20251028_120000.pkl'

# Alternative: Find the most recent files automatically
results_dir = Path('results/extracted')

# Find BEZ file
bez_files = sorted(results_dir.glob('BEZ_extracted_60.0dB_*.pkl'))
if bez_files:
    bez_file = str(bez_files[-1])  # Use most recent
    print(f"Found BEZ file: {bez_file}")

# Find Cochlea file
cochlea_files = sorted(results_dir.glob('COCHLEA_extracted_60.0dB_*.pkl'))
if cochlea_files:
    cochlea_file = str(cochlea_files[-1])  # Use most recent
    print(f"Found Cochlea file: {cochlea_file}")

# Verify files exist
bez_path = Path(bez_file)
cochlea_path = Path(cochlea_file)

print(f"\nBEZ file: {bez_path}")
print(f"  Exists: {bez_path.exists()}")
print(f"\nCochlea file: {cochlea_path}")
print(f"  Exists: {cochlea_path.exists()}")

# Configuration
fiber_type = 'hsr'  # Options: 'hsr', 'msr', 'lsr'
output_dir = 'results/correlations'

print(f"\nConfiguration:")
print(f"  Fiber type: {fiber_type.upper()}")
print(f"  Output directory: {output_dir}")

BEZ file: results/extracted/BEZ_extracted_60.0dB.pkl
  Exists: True

Cochlea file: results/extracted/Cochlea_extracted_60.0dB.pkl
  Exists: True

Configuration:
  Fiber type: HSR
  Output directory: results/correlations


## Step 2: Load Extracted Data

Load the extracted data files and examine their structure.

In [3]:
# Load data
print("Loading extracted data...\n")

bez_data, bez_meta = load_extracted_data(bez_file)
cochlea_data, cochlea_meta = load_extracted_data(cochlea_file)

print("=== BEZ Data ===")
print(f"dB level: {bez_meta['extraction_summary']['target_db']}")
print(f"Fiber types: {list(bez_data.keys())}")
print(f"Structure: [fiber_type][cf][freq][run]")

print("\n=== Cochlea Data ===")
print(f"dB level: {cochlea_meta['extraction_summary']['target_db']}")
print(f"Fiber types: {list(cochlea_data.keys())}")
print(f"Structure: [fiber_type][cf][freq]")

# Check data for selected fiber type
bez_fiber = bez_data[fiber_type]
cochlea_fiber = cochlea_data[fiber_type]

print(f"\n=== {fiber_type.upper()} Fiber Data ===")
print(f"BEZ CFs: {len(bez_fiber)} conditions")
print(f"Cochlea CFs: {len(cochlea_fiber)} conditions")

# Sample one condition
sample_cf = list(bez_fiber.keys())[0]
sample_freq = list(bez_fiber[sample_cf].keys())[0]
sample_runs = bez_fiber[sample_cf][sample_freq]

print(f"\nSample condition (CF={sample_cf}Hz, Freq={sample_freq}Hz):")
print(f"  BEZ runs: {len(sample_runs)}")
print(f"  Run shape: {sample_runs[0].shape if 0 in sample_runs else 'N/A'}")

Loading extracted data...

=== BEZ Data ===
dB level: 60.0
Fiber types: ['hsr', 'msr', 'lsr']
Structure: [fiber_type][cf][freq][run]

=== Cochlea Data ===
dB level: 60.0
Fiber types: ['hsr', 'msr', 'lsr']
Structure: [fiber_type][cf][freq]

=== HSR Fiber Data ===
BEZ CFs: 20 conditions
Cochlea CFs: 20 conditions

Sample condition (CF=125.0Hz, Freq=125.0Hz):
  BEZ runs: 10
  Run shape: (519,)


In [7]:
# Check available conditions in detail
print("=== Available Conditions in Extracted Data ===\n")

print(f"Fiber type: {fiber_type.upper()}\n")

# Check BEZ conditions
print("BEZ Data:")
print(f"  Number of CFs: {len(bez_fiber)}")
print(f"  CF values: {sorted(list(bez_fiber.keys()))}\n")

for cf in sorted(list(bez_fiber.keys())):
    freqs = sorted(list(bez_fiber[cf].keys()))
    print(f"  CF={cf}Hz has {len(freqs)} frequencies: {freqs}")

# Check Cochlea conditions
print("\nCochlea Data:")
print(f"  Number of CFs: {len(cochlea_fiber)}")
print(f"  CF values: {sorted(list(cochlea_fiber.keys()))}\n")

for cf in sorted(list(cochlea_fiber.keys())):
    freqs = sorted(list(cochlea_fiber[cf].keys()))
    print(f"  CF={cf}Hz has {len(freqs)} frequencies: {freqs}")

# Count total combinations
print("\nAll CF × Frequency combinations:")
all_conditions = []
for cf in sorted(list(bez_fiber.keys())):
    for freq in sorted(list(bez_fiber[cf].keys())):
        all_conditions.append((cf, freq))
        print(f"  {len(all_conditions)}. CF={cf}Hz, Freq={freq}Hz")

print(f"\nTotal unique combinations: {len(all_conditions)}")

# Check extraction summary
print("\n=== Extraction Summary ===")
print(f"BEZ: {bez_meta['extraction_summary']}")
print(f"\nCochlea: {cochlea_meta['extraction_summary']}")

=== Available Conditions in Extracted Data ===

Fiber type: HSR

BEZ Data:
  Number of CFs: 20
  CF values: [np.float64(125.0), np.float64(159.49726883), np.float64(198.39318028), np.float64(242.24859198), np.float64(291.69587485), np.float64(347.44803149), np.float64(410.30897734), np.float64(481.18513264), np.float64(561.09849255), np.float64(651.20136376), np.float64(752.79298009), np.float64(867.33823676), np.float64(996.48881333), np.float64(1142.10699008), np.float64(1306.29250097), np.float64(1491.41281075), np.float64(1700.13725242), np.float64(1935.4755176), np.float64(2200.82105458), np.float64(2500.0)]

  CF=125.0Hz has 20 frequencies: [np.float64(125.0), np.float64(159.49726883), np.float64(198.39318028), np.float64(242.24859198), np.float64(291.69587485), np.float64(347.44803149), np.float64(410.30897734), np.float64(481.18513264), np.float64(561.09849255), np.float64(651.20136376), np.float64(752.79298009), np.float64(867.33823676), np.float64(996.48881333), np.float64(11

In [4]:
# Debug: Inspect loaded data structure
import logging
logging.basicConfig(level=logging.INFO, format='%(message)s')

# Check BEZ data structure
print("=== BEZ Data Structure Check ===")
first_cf = list(bez_data[fiber_type].keys())[0]
first_freq = list(bez_data[fiber_type][first_cf].keys())[0]
runs = bez_data[fiber_type][first_cf][first_freq]

print(f"\nFirst condition: CF={first_cf}Hz, Freq={first_freq}Hz")
print(f"Number of runs: {len(runs)}")
print(f"Run keys: {list(runs.keys())}")

# Check if runs are different
if len(runs) >= 2:
    run_keys = sorted(runs.keys())
    run0 = runs[run_keys[0]]
    run1 = runs[run_keys[1]]
    
    print(f"\nRun {run_keys[0]}:")
    print(f"  Shape: {run0.shape}")
    print(f"  First 10 values: {run0[:10]}")
    print(f"  Mean: {run0.mean():.4f}")
    
    print(f"\nRun {run_keys[1]}:")
    print(f"  Shape: {run1.shape}")
    print(f"  First 10 values: {run1[:10]}")
    print(f"  Mean: {run1.mean():.4f}")
    
    print(f"\nArrays identical? {np.array_equal(run0, run1)}")
    print(f"Correlation: {np.corrcoef(run0, run1)[0, 1]:.4f}")

=== BEZ Data Structure Check ===

First condition: CF=125.0Hz, Freq=125.0Hz
Number of runs: 10
Run keys: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Run 0:
  Shape: (519,)
  First 10 values: [0.00000000e+00 0.00000000e+00 3.58520620e-09 2.31847562e-07
 2.44835269e-06 1.29017336e-05 4.68431351e-05 1.34326780e-04
 3.26387152e-04 7.00889564e-04]
  Mean: 6.4683

Run 1:
  Shape: (519,)
  First 10 values: [0.00000000e+00 0.00000000e+00 3.25927836e-09 2.11303848e-07
 2.25897735e-06 1.20087669e-05 4.35590231e-05 1.23832125e-04
 2.97070235e-04 6.28628450e-04]
  Mean: 6.2891

Arrays identical? False
Correlation: 0.9992


## Step 3: Calculate Within-BEZ Correlations

Calculate pairwise correlations between all BEZ runs for each CF × Frequency 
combination.

In [5]:
# Calculate within-BEZ correlations
print(f"Calculating within-BEZ correlations for {fiber_type.upper()}...\n")

within_bez, run_pairs = calculate_within_bez_correlations(
    bez_fiber, fiber_type
)

# Summarize results
total_pairs = sum(
    len(within_bez[cf][freq]) 
    for cf in within_bez 
    for freq in within_bez[cf]
)

print(f"Total pairwise correlations: {total_pairs}")

# Show sample correlations
print(f"\nSample correlations (CF={sample_cf}Hz, Freq={sample_freq}Hz):")
sample_corrs = within_bez[sample_cf][sample_freq]
print(f"  Number of pairs: {len(sample_corrs)}")
print(f"  Mean correlation: {np.mean(sample_corrs):.3f}")
print(f"  Std correlation: {np.std(sample_corrs):.3f}")
print(f"  Range: [{np.min(sample_corrs):.3f}, "
      f"{np.max(sample_corrs):.3f}]")

# Calculate overall statistics
all_within_corrs = np.concatenate([
    within_bez[cf][freq] 
    for cf in within_bez 
    for freq in within_bez[cf]
])

print(f"\n=== Overall Within-BEZ Statistics ===")
print(f"Mean correlation: {np.mean(all_within_corrs):.3f}")
print(f"Median correlation: {np.median(all_within_corrs):.3f}")
print(f"Std correlation: {np.std(all_within_corrs):.3f}")
print(f"Range: [{np.min(all_within_corrs):.3f}, "
      f"{np.max(all_within_corrs):.3f}]")


DEBUG: Calculating correlations for hsr
Number of CFs: 20
CF=125.0Hz, Freq=125.0Hz: 10 runs
  First pair (run 0 vs 1):
    Run i shape: (519,)
    Run j shape: (519,)
    First 5 values i: [0.00000000e+00 0.00000000e+00 3.58520620e-09 2.31847562e-07
 2.44835269e-06]
    First 5 values j: [0.00000000e+00 0.00000000e+00 3.25927836e-09 2.11303848e-07
 2.25897735e-06]
    Arrays equal? False
  Correlations: min=0.999, max=1.000, mean=0.999
CF=125.0Hz, Freq=159.49726883Hz: 10 runs
  First pair (run 0 vs 1):
    Run i shape: (519,)
    Run j shape: (519,)
    First 5 values i: [0.00000000e+00 0.00000000e+00 2.60742269e-09 1.69564563e-07
 1.83979108e-06]
    First 5 values j: [0.00000000e+00 0.00000000e+00 1.30371134e-09 8.54341371e-08
 9.63916319e-07]
    Arrays equal? False
  Correlations: min=0.998, max=1.000, mean=0.999
CF=125.0Hz, Freq=198.39318028Hz: 10 runs
  First pair (run 0 vs 1):
    Run i shape: (519,)
    Run j shape: (519,)
    First 5 values i: [0.00000000e+00 0.00000000e+00 1

Calculating within-BEZ correlations for HSR...



  Correlations: min=0.998, max=1.000, mean=0.999
CF=159.49726883Hz, Freq=198.39318028Hz: 10 runs
  First pair (run 0 vs 1):
    Run i shape: (519,)
    Run j shape: (519,)
    First 5 values i: [0.00000000e+00 0.00000000e+00 1.30371134e-09 8.47822814e-08
 9.20547395e-07]
    First 5 values j: [0.00000000e+00 0.00000000e+00 1.95556702e-09 1.27499350e-07
 1.40136481e-06]
    Arrays equal? False
  Correlations: min=0.999, max=1.000, mean=1.000
CF=159.49726883Hz, Freq=242.24859198Hz: 10 runs
  First pair (run 0 vs 1):
    Run i shape: (519,)
    Run j shape: (519,)
    First 5 values i: [0.00000000e+00 0.00000000e+00 1.30371134e-09 8.47822814e-08
 9.20873323e-07]
    First 5 values j: [0.00000000e+00 0.00000000e+00 3.91113403e-09 2.53694988e-07
 2.71859919e-06]
    Arrays equal? False
  Correlations: min=0.998, max=1.000, mean=0.999
CF=159.49726883Hz, Freq=291.69587485Hz: 10 runs
  First pair (run 0 vs 1):
    Run i shape: (519,)
    Run j shape: (519,)
    First 5 values i: [0.00000000e+0

Total pairwise correlations: 18000

Sample correlations (CF=125.0Hz, Freq=125.0Hz):
  Number of pairs: 45
  Mean correlation: 0.999
  Std correlation: 0.000
  Range: [0.999, 1.000]

=== Overall Within-BEZ Statistics ===
Mean correlation: 0.999
Median correlation: 0.999
Std correlation: 0.001
Range: [0.990, 1.000]


## Step 4: Calculate Cochlea-BEZ Correlations

Calculate correlation between Cochlea response and mean BEZ response for each 
CF × Frequency combination.

In [6]:
# Calculate Cochlea-BEZ correlations
print(f"Calculating Cochlea-BEZ correlations for "
      f"{fiber_type.upper()}...\n")

cochlea_bez = calculate_cochlea_bez_correlation(
    cochlea_fiber, bez_fiber
)

# Summarize results
total_comparisons = sum(
    1 for cf in cochlea_bez for freq in cochlea_bez[cf]
)

print(f"Total CF × Frequency comparisons: {total_comparisons}")

# Show sample correlation
if sample_cf in cochlea_bez and sample_freq in cochlea_bez[sample_cf]:
    sample_cb_corr = cochlea_bez[sample_cf][sample_freq]
    print(f"\nSample correlation (CF={sample_cf}Hz, "
          f"Freq={sample_freq}Hz):")
    print(f"  Cochlea-BEZ correlation: {sample_cb_corr:.3f}")

# Calculate overall statistics
all_cb_corrs = np.array([
    cochlea_bez[cf][freq] 
    for cf in cochlea_bez 
    for freq in cochlea_bez[cf]
])

print(f"\n=== Overall Cochlea-BEZ Statistics ===")
print(f"Mean correlation: {np.mean(all_cb_corrs):.3f}")
print(f"Median correlation: {np.median(all_cb_corrs):.3f}")
print(f"Std correlation: {np.std(all_cb_corrs):.3f}")
print(f"Range: [{np.min(all_cb_corrs):.3f}, "
      f"{np.max(all_cb_corrs):.3f}]")

Calculating Cochlea-BEZ correlations for HSR...

Total CF × Frequency comparisons: 4

Sample correlation (CF=125.0Hz, Freq=125.0Hz):
  Cochlea-BEZ correlation: -0.005

=== Overall Cochlea-BEZ Statistics ===
Mean correlation: -0.168
Median correlation: -0.164
Std correlation: 0.150
Range: [-0.340, -0.005]


## Step 5: Calculate Percentile Rankings

Determine where each Cochlea-BEZ correlation falls in the distribution of 
within-BEZ correlations.

In [ ]:
# Calculate percentile ranks for each condition
print("Calculating percentile rankings...\n")

percentile_ranks = {}

for cf_val in cochlea_bez:
    percentile_ranks[cf_val] = {}
    
    for freq_val in cochlea_bez[cf_val]:
        cb_corr = cochlea_bez[cf_val][freq_val]
        wb_corrs = within_bez[cf_val][freq_val]
        
        percentile = calculate_percentile_rank(cb_corr, wb_corrs)
        percentile_ranks[cf_val][freq_val] = percentile

# Show sample percentile
if sample_cf in percentile_ranks and (
    sample_freq in percentile_ranks[sample_cf]
):
    sample_percentile = percentile_ranks[sample_cf][sample_freq]
    print(f"Sample percentile (CF={sample_cf}Hz, "
          f"Freq={sample_freq}Hz):")
    print(f"  Percentile rank: {sample_percentile:.1f}%")
    print(f"  Interpretation: Cochlea-BEZ correlation is higher than "
          f"{sample_percentile:.1f}% of within-BEZ correlations")

# Calculate overall percentile
all_percentiles = np.array([
    percentile_ranks[cf][freq] 
    for cf in percentile_ranks 
    for freq in percentile_ranks[cf]
])

print(f"\n=== Percentile Rankings Summary ===")
print(f"Mean percentile: {np.mean(all_percentiles):.1f}%")
print(f"Median percentile: {np.median(all_percentiles):.1f}%")
print(f"Conditions above 50th percentile: "
      f"{np.sum(all_percentiles > 50)} / {len(all_percentiles)}")
print(f"Conditions above 75th percentile: "
      f"{np.sum(all_percentiles > 75)} / {len(all_percentiles)}")

## Step 6: Run Complete Analysis

Use the main analysis function to get comprehensive results.

In [ ]:
# Run complete analysis
results = analyze_correlations(
    bez_file,
    cochlea_file,
    fiber_type=fiber_type
)

print("\n✓ Complete analysis finished")

## Step 7: Visualize Results

Plot comparison of within-BEZ and Cochlea-BEZ correlation distributions.

In [ ]:
# Plot correlation comparison
plot_correlation_comparison(results, output_dir=output_dir)

print(f"\n✓ Plots saved to: {output_dir}")

## Step 8: Analyze Results by CF and Frequency

Examine correlations broken down by CF and frequency combinations.

In [ ]:
# Create detailed breakdown
print("=== Correlation Analysis by CF ===\n")

for cf_val in sorted(list(results['cochlea_bez'].keys()))[:5]:
    print(f"CF = {cf_val:.0f} Hz:")
    
    cf_within = []
    cf_cb = []
    
    for freq_val in results['cochlea_bez'][cf_val]:
        wb = results['within_bez'][cf_val][freq_val]
        cb = results['cochlea_bez'][cf_val][freq_val]
        
        cf_within.extend(wb)
        cf_cb.append(cb)
    
    print(f"  Within-BEZ: mean={np.mean(cf_within):.3f}, "
          f"std={np.std(cf_within):.3f}")
    print(f"  Cochlea-BEZ: mean={np.mean(cf_cb):.3f}, "
          f"std={np.std(cf_cb):.3f}")
    print()

## Step 9: Save Results

Save the complete analysis results to a pickle file for later use.

In [ ]:
# Save results
output_path = Path(output_dir)
output_path.mkdir(parents=True, exist_ok=True)

output_file = output_path / f'correlation_results_{fiber_type}.pkl'

with open(output_file, 'wb') as f:
    pickle.dump(results, f)

print(f"✓ Results saved to: {output_file}")

# Print file size
file_size = output_file.stat().st_size / 1024  # KB
print(f"  File size: {file_size:.1f} KB")

## Step 10: Analyze Other Fiber Types (Optional)

Run the analysis for MSR and LSR fiber types as well.

In [ ]:
# Analyze all fiber types
all_results = {}

for ft in ['hsr', 'msr', 'lsr']:
    print(f"\n{'='*60}")
    print(f"Analyzing {ft.upper()} fiber type...")
    print(f"{'='*60}")
    
    try:
        results_ft = analyze_correlations(
            bez_file,
            cochlea_file,
            fiber_type=ft
        )
        
        all_results[ft] = results_ft
        
        # Plot for this fiber type
        plot_correlation_comparison(results_ft, output_dir=output_dir)
        
        print(f"✓ {ft.upper()} analysis complete")
        
    except Exception as e:
        print(f"✗ Error analyzing {ft.upper()}: {e}")

print("\n=== All Fiber Types Analyzed ===")




1. **Within-BEZ variability**: Shows consistency across multiple runs of the 
   same model
2. **Cochlea-BEZ agreement**: Measures how well Cochlea predicts mean BEZ 
   response
3. **Percentile ranking**: Indicates whether Cochlea-BEZ correlation is 
   within the range of within-BEZ variability

**Interpretation:**
- **High percentile (>75%)**: Cochlea correlates better with mean BEZ than 
  most BEZ runs correlate with each other
- **Medium percentile (25-75%)**: Cochlea-BEZ correlation is typical of 
  within-BEZ variability
- **Low percentile (<25%)**: Cochlea-BEZ correlation is weaker than typical 
  within-BEZ correlations





- Within-BEZ correlations are **very high (~0.9992)** - this is expected




